In [23]:
#We need these in this file:
import pandas as pd
from sklearn import svm
from sklearn.metrics import classification_report,accuracy_score
import numpy as np # linear algebra
import get_lodging_scores
import load_read_name_extractor as lrne
import SVM_classifier_general as svm_general
import pickle
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV

In [32]:
data_paths = [
"Features\harlick_features_96x192_contrast_entopy.npy",
"Features\harlick_features_96x192_entopy.npy",
"Features\harlick_features_96x192_contrast.npy",
"Features\harlick_features_96x192_gaussian_filter_sigma=15.npy",
"Features\harlick_features_96x192_14_features.npy",
"Features\img_names_96x192_contrast_entopy.npy",
"Features\img_names_96x192_entopy.npy",
"Features\img_names_96x192_contrast.npy",
"Features\img_names_96x192_gaussian_filter_sigma=15.npy",
"Features\img_names_96x192_14_features.npy"
]

In [40]:
features_paths = data_paths[0:5]
names_paths = data_paths[5:10]
#features_paths = ["Features\harlick_features_96x192_test_af_labels_som_features.npy"]
#names_paths = ["Features\img_names_96x192_test_af_labels_som_features.npy"]

In [41]:
features_paths

['Features\\harlick_features_96x192_contrast_entopy.npy',
 'Features\\harlick_features_96x192_entopy.npy',
 'Features\\harlick_features_96x192_contrast.npy',
 'Features\\harlick_features_96x192_gaussian_filter_sigma=15.npy',
 'Features\\harlick_features_96x192_14_features.npy']

In [42]:
names_paths

['Features\\img_names_96x192_contrast_entopy.npy',
 'Features\\img_names_96x192_entopy.npy',
 'Features\\img_names_96x192_contrast.npy',
 'Features\\img_names_96x192_gaussian_filter_sigma=15.npy',
 'Features\\img_names_96x192_14_features.npy']

In [34]:
def save_results(results, path):
    path = path.replace("Features\\", "")
    path = "results/" + path.replace(".npy", ".txt")
    with open(path, 'a') as f:
        f.write(results)
        f.write("\n")

In [35]:
def load_labels(bin_size):
    labels = get_lodging_scores.get_labels(bin_size)
    #COnverts into set (no dublicates in set)
    my_set = set(map(tuple, labels))
    # convert set back to list of arrays
    labels = list(map(np.array, my_set))
    return labels

In [36]:
def partition_data(data_frame, features_aar, percentage):
    partition = int(len(features_aar)*percentage/100)
    x_train, x_test = data_frame[:partition,:-1],  data_frame[partition:,:-1]
    y_train, y_test = data_frame[:partition,-1:].ravel() , data_frame[partition:,-1:].ravel()
    return x_train, x_test, y_train, y_test

In [37]:
def save_model(filename, clf):
    filename = filename.replace("Features\\", "")
    filename = "model/" + filename.replace(".npy", ".sav")
    pickle.dump(clf, open(filename, 'wb'))

In [38]:
def run_mult_models(features_paths, names_paths, bin_size = 10, svm_kernel="rbf", class_weights = None):
    for i in range (len(features_paths)):
        #laod features and names
        features_load = np.load(features_paths[i], allow_pickle=True)
        names_load = np.load(names_paths[i], allow_pickle=True)

        #load labels
        labels = load_labels(bin_size)

        #convert names into [flight_folder, ROI]
        names = lrne.it_name_extract_labels_from_img_jpeg(names_load)

        #create svm
        clf = svm.SVC(kernel=svm_kernel, class_weight=class_weights)

        #match labels with feature names
        data_frame, error = svm_general.match_pic_label_to_names(features_load, labels, names)
        #data_frame_names, error_names = match_pic_label_names(features_load, labels, names) - finds images that has no labels

        #shuffle data and convert to array
        np.random.shuffle(data_frame)
        data_frame = np.array(data_frame)

        #partition data
        percentage = 80
        features_aar = np.array(features_load)
        x_train, x_test, y_train, y_test = partition_data(data_frame, features_aar, percentage)

        #fit model with training data
        clf.fit(x_train,y_train)

        #predict data
        y_pred = clf.predict(x_test)

        #Save the found model
        n_bins = int(np.floor(100/bin_size))
        path = "Kernel_" + svm_kernel + "_Nbins_" + str(n_bins) + "_ClassWeights_" + str(class_weights) + "_" + features_paths[i]
        save_model(features_paths[i], clf)

        #Save the results
        results = "Accuracy: "+str(accuracy_score(y_test, y_pred)) + "\n" + classification_report(y_test, y_pred)
        save_results(results, path)
        
        
    

In [43]:
run_mult_models(features_paths, names_paths, bin_size = 10, svm_kernel="rbf", class_weights="balanced")

c:\Users\Bruger\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Bruger\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Bruger\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Bruger\miniconda3\lib\site-packages\sklearn\metr

In [ ]:
features_load = np.load('features/*', allow_pickle=True)
names_load = np.load('features/img_names_64_128_hog_cells_8x8_block_2x2.npy', allow_pickle=True)
print(len(features_load))
print(len(names_load))

In [ ]:
labels = get_lodging_scores.get_labels(33)
#[ROI, Visual SCORE, FLIGHTFolder, BIN]
print(features_load[0])
print(labels[0])

[0.20888097 0.16330611 0.04696022 ... 0.06301632 0.05749686 0.03150874]
(61, 50, '160607_Min', 2.0)


In [ ]:
my_set = set(map(tuple, labels))
# convert set back to list of arrays
labels = list(map(np.array, my_set))

In [ ]:
print(len(labels))

4936


In [ ]:
names = lrne.it_name_extract_labels_from_img_jpeg(names_load)

In [ ]:
clf = svm.SVC(kernel="rbf", class_weight="balanced")
features_aar = np.array(features_load)
data_frame, error = SVM_GENERAL.match_pic_label_to_names(features_load, labels, names)
#data_frame_names, error_names = match_pic_label_names(features_load, labels, names) - finds images that has no labels
np.random.shuffle(data_frame)
data_frame = np.array(data_frame)


In [ ]:
percentage = 80
partition = int(len(features_aar)*percentage/100)

In [ ]:
x_train, x_test = data_frame[:partition,:-1],  data_frame[partition:,:-1]
y_train, y_test = data_frame[:partition,-1:].ravel() , data_frame[partition:,-1:].ravel()
print(y_train)

clf.fit(x_train,y_train)

['1.0' '1.0' '1.0' ... '2.0' '1.0' '1.0']


SVC(class_weight='balanced')

In [ ]:
print(clf)

SVC(class_weight='balanced')


In [ ]:
y_pred = clf.predict(x_test)

In [ ]:
filename = 'SVM_linear_64_128_3bins_cells_8x8_block_2x2_balanced.sav'
pickle.dump(clf, open(filename, 'wb'))

In [ ]:
#loaded_model = pickle.load(open(filename, 'rb'))
#result = loaded_model.score(X_test, Y_test)

In [ ]:
print("Accuracy: "+str(accuracy_score(y_test, y_pred)))
print('\n')
print(classification_report(y_test, y_pred))

Accuracy: 0.717683557394002


              precision    recall  f1-score   support

         1.0       0.86      0.80      0.83       687
         2.0       0.39      0.52      0.45       192
         3.0       0.60      0.50      0.55        88

    accuracy                           0.72       967
   macro avg       0.62      0.61      0.61       967
weighted avg       0.74      0.72      0.73       967

